# Connect to the mid_term_project PostgreSQL database

In [258]:
import psycopg2  # PostgreSQL database adapter
from psycopg2 import sql  # SQL string composition
import pandas as pd

[psycopg2 documentation](https://www.psycopg.org/docs/)

Use database credentials from [Compass](https://data.compass.lighthouselabs.ca/days/w05d5/activities/823)

In [259]:
def postgresql_connection():
    """
    Create a new database session
    
        Parameters:
            None
        
        Returns:
            connection (psycopg2 connection object): A PostgreSQL connection
    
    The format of database_credentials.txt corresponds to psycopg2.connect()
    parameter format.
        Example: "dbname=test user=postgres password=secret"
    
    Credentials are retrievable from Compass
    (https://data.compass.lighthouselabs.ca/days/w05d5/activities/823)
    """
    
    # database_credentials.txt is in .gitignore
    db_credentials = open('../data/database_credentials.txt', 'r').read()
    
    connection = psycopg2.connect(db_credentials)
    
    return connection

In [260]:
def dataframe_to_csv(df, csv_path: str):
    """
    Save Pandas Dataframe to csv file
        
    Parameter
    ---------
    df : Pandas Dataframe
        query results
    csv_path : string
        filepath
        
    Returns
    -------
    None
    
    This works well after loading a PostgreSQL query result into memory.
    """
    
    # Write csv with header and without index column
    df.to_csv(csv_path, header=True, index=False)
    
    return None

In [257]:
def postgresql_to_csv(sql_statement: 'str', csv_path: 'str'):
    """
    Save PostgreSQL query results to csv file
    
    Parameters
    ----------
    sql_statement : string
        SQL statement
    csv_path : string
        filepath
    
    Returns
    -------
    None

    Adapted from ObjectRocket
    https://kb.objectrocket.com/postgresql/from-postgres-to-csv-with-python-910
    """
    
    # remove semicolon from query string
    S = sql_statement.replace(';', '')
    
    # SQL composition to prevent SQL injection
    SQL_for_file_output = (
        sql.SQL('COPY ({}) TO STDOUT WITH CSV HEADER')
        .format(sql.SQL(S))
    )
    
    # Write query results to csv
    with open(csv_path, 'w') as f_output:
        cursor.copy_expert(SQL_for_file_output, f_output)
    
    return None

In [170]:
def execute_sql_statement(connection, query: 'str',
                          variables: 'tuple | None' = None,
                          save_to_csv: 'bool' = False,
                          csv_path: 'str | None' = None):
    """
    Returns a PostgreSQL query as a Pandas Dataframe
    
    Parameters
    ----------
        connection : (psycopg2 connection object)
            A PostgreSQL connection
        query : string
            SQL query
        variables : tuple or None, default None
            Parameters to pass to SQL query
        save_to_csv : bool, default False
            Write query result to csv
        csv_path : string or None, default None
            Filepath to save csv output
            
    Returns
    -------
        df : dataframe of query results
    """

    cursor = connection.cursor()  # client side cursor

    cursor.execute(query=query, vars=variables)  # execute sql statement
    
    column_names = [desc[0] for desc in cursor.description]
    
    rows = cursor.fetchall()  # Fetch all (remaining) rows of a query result
    
    df = pd.DataFrame(rows, columns=column_names)
    
    if (save_to_csv) & (csv_path != None):
        dataframe_to_csv(df=df, path=csv_path)
    
    return df

In [254]:
connection = postgresql_connection()
cursor = connection.cursor()

In [255]:
SQL_statement = """
SELECT *
    FROM flights
    LIMIT 10;
"""

In [256]:
postgresql_to_csv(sql_statement=SQL_statement, csv_path='../data/filename.csv')

In [155]:
df = execute_sql_statement(connection=connection, query=SQL,
                           save_to_csv=True, csv_path='../data/test.csv')

In [156]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-08-11,WN,WN,WN,2779,WN,N7831B,2779,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,2019-08-11,WN,WN,WN,3413,WN,N743SW,3413,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2,2019-08-11,WN,WN,WN,4131,WN,N7738A,4131,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
3,2019-08-11,WN,WN,WN,4159,WN,N8538V,4159,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,2019-08-11,WN,WN,WN,4254,WN,N427WN,4254,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
5,2019-08-11,WN,WN,WN,4421,WN,N7850B,4421,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
6,2019-08-11,WN,WN,WN,4470,WN,N270WN,4470,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
7,2019-08-11,WN,WN,WN,4933,WN,N733SA,4933,10800,BUR,...,296.0,12.0,0.0,2.0,0.0,54.0,None,None,None,None
8,2019-08-11,WN,WN,WN,5018,WN,N566WN,5018,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None
9,2019-08-11,WN,WN,WN,5601,WN,N8613K,5601,10800,BUR,...,296.0,NaN,NaN,NaN,NaN,NaN,None,None,None,None


# Database description 

In [69]:
def get_table_data_types(connection, table_name: str):
    """
    Returns a summary table of the columns and corresponding datatypes
    in a given PostgreSQL database table.
    
        Parameters:
            connection (psycopg2 connection object): A PostgreSQL connection
            table_name (str): name of the table
        
        Returns:
            (pandas.Dataframe): 
    """
    
    cursor = connection.cursor()  # client side cursor
    
    # Properly formated SQL statement that prevents SQL injection
    sql = f"""
    SELECT
       column_name, 
       data_type 
    FROM 
       information_schema.columns
    WHERE 
       table_name = (%s);
    """
        
    cursor.execute(query=sql, vars=(table_name,))  # execute sql statement
    
    rows = cursor.fetchall()  # Fetch all (remaining) rows of a query result
    
    df = pd.DataFrame(rows, columns=['column', 'data_type'])
    df.columns.name = table_name
    return df

In [49]:
df = get_table_data_types(con, 'flights')

In [51]:
df

flights,column,dtype
0,fl_date,text
1,mkt_unique_carrier,text
2,branded_code_share,text
3,mkt_carrier,text
4,mkt_carrier_fl_num,integer
5,op_unique_carrier,text
6,tail_num,text
7,op_carrier_fl_num,integer
8,origin_airport_id,integer
9,origin,text


[How to derive summary statistics using PostgreSQL](https://towardsdatascience.com/how-to-derive-summary-statistics-using-postgresql-742f3cdc0f44)

# Close database cursor and connection

In [26]:
cur.close()
con.close()